## Barrel shifter (cyhdl emulation library)

This variant uses mixed myIRL and cythonized myhdl code segments. Note that the procedural generation is quite a bit faster, once the module is cythonized.

This is using a `cyhdl` emulation and the `pyximport` mechanisms to ad-hoc cythonize Python code. The complation is taking place upon import.

We import the barrelshifter library:

In [7]:
from library.barrelshifter import *

[Edit barrelshifter.py](library/barrelshifter.py)

Then we create a barrel shifter generator with data width 32 (power 5):

In [8]:
W_POWER = 5

TEST_VALUES = [
    (0xdead, 8, 0xdead << 8),
    (0x8f01, 15, 0x8f01 << 15),
]

b = BarrelShifterGenerator(W_POWER)

Then we instance it first time in the test bench below. Note the instancing time on this first run.

In [9]:
from myirl.test.common_test import gen_osc
from cyhdl import *

@block
def top_bs(b):
    clk = ClkSignal()
    ce = Signal(bool())
    val, result = [ Signal(intbv(0xaa00)[2 ** W_POWER:]) for i in range(2) ]
    s = Signal(intbv()[W_POWER:])
    
    inst = [
        b.barrel_shifter(clk, ce, val, s, result, False),
        gen_osc(clk, 2)
    ]

    @instance
    def stim():
        for item in TEST_VALUES:
            ce.next = False
            s.next = item[1]
            val.next = item[0]
            yield(clk.posedge)
            ce.next = True
            yield(clk.posedge)
            yield(clk.posedge)

            print(result)
            assert result == item[2]
            
        raise StopSimulation

    inst += [ stim ]
    return inst
 
@utils.timer
def test(b):    
    return top_bs(b)

design = test(b)

 Declare obj 'barrel_shifter' in context '(LIB: BarrelShifterGenerator 'bs')'(<class '_cython_inline_d4d0c88a151ed2920fb2029ff3e31d497bd207d2.__invoke.<locals>.BarrelShifterGenerator'>) 
 Declare obj 'shifter_stage' in context '(LIB: BarrelShifterGenerator 'bs')'(<class '_cython_inline_d4d0c88a151ed2920fb2029ff3e31d497bd207d2.__invoke.<locals>.BarrelShifterGenerator'>) 
 Declare obj 'cshift' in context '(LIB: BarrelShifterGenerator 'bs')'(<class '_cython_inline_d4d0c88a151ed2920fb2029ff3e31d497bd207d2.__invoke.<locals>.BarrelShifterGenerator'>) 
 Module bs: Existing instance cshift, rename to cshift_1 
 Module bs: Existing instance shifter_stage, rename to shifter_stage_1 
 Module bs: Existing instance shifter_stage, rename to shifter_stage_2 
 Module bs: Existing instance shifter_stage, rename to shifter_stage_3 
 Module bs: Existing instance shifter_stage, rename to shifter_stage_4 
Finished test in 0.3339 secs


Running the test bench again, the duration is a little less than on the first run, because this module was already compiled.

In [10]:
design = test(b)

 Module top_bs: Existing instance top_bs, rename to top_bs_1 
Finished test in 0.0885 secs


Finally, the quick & dirty verification:

In [5]:
from myirl import targets
from myirl.test.common_test import run_ghdl

f = design.elab(targets.VHDL, elab_all = True)
f += b.elab(targets.VHDL)
# print(f)
run_ghdl(f, design, vcdfile = 'bs.vcd', debug = True)

 Writing 'top_bs_1' to file /tmp/myirl_top_bs_gyiuwx6a/top_bs_1.vhdl 
 Creating library file module_defs.vhdl 
 Writing 'shifter_stage_4' to file shifter_stage_4.vhdl 
 Writing 'shifter_stage_3' to file shifter_stage_3.vhdl 
 Writing 'shifter_stage_2' to file shifter_stage_2.vhdl 
 Writing 'shifter_stage_1' to file shifter_stage_1.vhdl 
 Writing 'cshift_1' to file cshift_1.vhdl 
 Writing 'cshift' to file cshift.vhdl 
 Writing 'shifter_stage' to file shifter_stage.vhdl 
 Writing 'barrel_shifter' to file barrel_shifter.vhdl 
 Not emitting design types library 
WORK DIR of instance [Instance top_bs_1 I/F: [// ID: top_bs_0 to top_bs]] /tmp/myirl_top_bs_gyiuwx6a/
==== COSIM stdout ====
0x00DEAD00
0x47808000
simulation stopped @22ns



0

In [6]:
# !cat -n /tmp/barrel_shifter.vhdl